In [ ]:
!pip install datasets transformers sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Loading the dataset

In [ ]:
# Importing the load_dataset function from the datasets library
from datasets import load_dataset

# Loading the "rotten_tomatoes" dataset and storing it in the variable 'data'
data = load_dataset("rotten_tomatoes")

# Printing the content of the loaded dataset to the console
print(data)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})


In [ ]:
data['train'][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [ ]:
data["train"][7000]

{'text': "it won't be long before you'll spy i spy at a video store near you .",
 'label': 0}

# Load Embeddings Model

In [ ]:
# Importing the SentenceTransformer class from the sentence_transformers library
from sentence_transformers import SentenceTransformer

# Initializing the model by loading the pre-trained 'all-mpnet-base-v2' model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Generate Embeddings for Labels

In [ ]:
# Encoding a list of sentences ("Negative movie review" and "Positive movie review")
# using the loaded model to obtain their embeddings (numerical representations)
label_embeddings = model.encode(["Negative movie review",  "Positive movie review"])

# Generate Embeddings for Test Data (Text)

In [ ]:
# Encoding the "text" column from the "test" split of the dataset `data` using the model.
# This will generate embeddings for each sentence in the "test" set.
# The `show_progress_bar=True` argument will display a progress bar during encoding.
test_embeddings = model.encode(data["test"]["text"], show_progress_bar=True)

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

# Zero-Shot Classification

In [ ]:
# Importing the cosine_similarity function from sklearn.metrics.pairwise
# to compute the cosine similarity between two sets of embeddings
from sklearn.metrics.pairwise import cosine_similarity

# Importing numpy for numerical operations (e.g., finding the index of the max value)
import numpy as np

# Compute the cosine similarity between the test embeddings and the label embeddings.
# The result is a similarity matrix where each row corresponds to a test sentence,
# and each column corresponds to one of the label embeddings ("Negative" or "Positive").
sim_matrix = cosine_similarity(test_embeddings, label_embeddings)

# For each document in the test set, find the index of the label with the highest cosine similarity.
# This gives the predicted label (0 for "Negative", 1 for "Positive").
predictions = np.argmax(sim_matrix, axis=1)

# Compute the Classification Report / Confusion Matrix

In [ ]:
# Importing the classification_report function from sklearn.metrics
# to evaluate the model's performance by generating precision, recall, and F1-score metrics
from sklearn.metrics import classification_report

# Extracting the true labels (ground truth) from the 'label' column in the "test" split of the dataset
y_true = data['test']["label"]

# Generating a classification report to evaluate the model's predictions against the true labels
# The report includes precision, recall, and F1-score for each class (Negative and Positive)
report = classification_report(y_true, predictions,
                               target_names=["Negative Movie Review", "Positive Movie Review"])

# Printing the classification report to the console
print(report)


                       precision    recall  f1-score   support

Negative Movie Review       0.88      0.65      0.75       533
Positive Movie Review       0.72      0.91      0.81       533

             accuracy                           0.78      1066
            macro avg       0.80      0.78      0.78      1066
         weighted avg       0.80      0.78      0.78      1066

